In [59]:
import requests
import pandas as pd
from math import log, sqrt, asin, exp, sin, cos, pi
from numpy import sign

Z_975 = 1.96
SP_ZERO_BLACKLIST = ['untransformed', 'logit']
MIN_DENOMINATOR = 250
pd.options.mode.chained_assignment = None

In [60]:
url = 'http://127.0.0.1:5000/airtable_scraper/records'
response = requests.get(url)

In [61]:
if response.status_code == 200:
    data = pd.DataFrame(response.json()['records'])

In [62]:
def calcTransformPrevalence(p, N, method):
    if method == 'untransformed': 
        return p
    elif method == 'logit': 
        return log(p / (1 - p))
    elif method == 'arcsin': 
        return asin(sqrt(p))
    elif method == 'double_arcsin_approx': 
        n = N * p
        return asin(sqrt(n / (N + 1))) + asin(sqrt((n + 1) / (N + 1)))
    elif method == 'double_arcsin_precise':
        n = N * p
        return 0.5 * (asin(sqrt(n / (N + 1))) + asin(sqrt((n + 1) / (N + 1))))

In [63]:
def calcTransformVariance(p, N, method):
    if method == 'untransformed': 
        return p * (1 - p) / N
    elif method == 'logit': 
        return 1 / (N * p) + 1 / (N * (1- p))
    elif method == 'arcsin': 
        return 1 / (4 * N)
    elif method == 'double_arcsin_approx': 
        return 1 / (N + 0.5)
    elif method == 'double_arcsin_precise': 
        return 1 / (4 * N + 2)

In [64]:
def backTransformPrevalence(t, n, method):
    if method == 'untransformed': 
        if t < 0: 
            return 0
        elif t > 1: 
            return 1
        else: 
            return t
    elif method == 'logit': 
        return exp(t) / (exp(t) + 1)
    elif method == 'arcsin': 
        if t < 0: 
            return 0
        elif t > pi / 2: 
            return 1
        else:
            return sin(t) ** 2
    elif method == 'double_arcsin_approx': 
        if t < 0: 
            return 0
        elif t > pi: 
            return 1
        else:
            return sin(t / 2) ** 2
    elif method == 'double_arcsin_precise': 
        if t < 0: 
            return 0
        elif t > pi / 4: 
            return 1
        else:
            return 0.5 * (1 - sign(cos(t)) * sqrt(1 - (sin(2 * t) + (sin(2 * t) - 2 * sin(2 * t)) / n) ** 2))

In [80]:
def calcBetweenStudyVariance(records):
    
    Q = sum(records['FIXED_WEIGHT'] * (records['TRANSFORMED_PREVALENCE'] - records['TRANSFORMED_POOLED_PREVALENCE']) ** 2)
    estimator_numerator = Q - (records.shape[0] - 1)
    estimator_denominator = sum(records['FIXED_WEIGHT']) - sum((records['FIXED_WEIGHT']) ** 2) / sum(records['FIXED_WEIGHT'])
    if estimator_denominator == 0:
        return 0
    else:
        return max(0, estimator_numerator/estimator_denominator)    

In [93]:
def metaprop(records, transformation_method = 'double_arcsin_precise', random_effects = True):
    filteredRecords = records[(records['SERUM_POS_PREVALENCE'].notna()) &
                            (records['DENOMINATOR'].notna()) &
                            (records['DENOMINATOR'] > 0) &
                            (records['SERUM_POS_PREVALENCE'] != 0 if transformation_method in SP_ZERO_BLACKLIST else True)]
    
    valid_records = (filteredRecords['DENOMINATOR'] >= MIN_DENOMINATOR).any()
    
    if valid_records:
        filteredRecords = filteredRecords[filteredRecords['DENOMINATOR'] >= MIN_DENOMINATOR]
    else:
        pass
    
    n_studies = filteredRecords.shape[0]
    if n_studies == 0: 
        return None 
    
    filteredRecords['TRANSFORMED_PREVALENCE'] = filteredRecords.apply(lambda row: calcTransformPrevalence(row['SERUM_POS_PREVALENCE'], 
                                                                                                          row['DENOMINATOR'], 
                                                                                                          transformation_method),
                                                                     axis = 1)
    filteredRecords['TRANSFORMED_VARIANCE'] = filteredRecords.apply(lambda row: calcTransformVariance(row['SERUM_POS_PREVALENCE'], 
                                                                                                          row['DENOMINATOR'], 
                                                                                                          transformation_method),
                                                                     axis = 1)
    
    filteredRecords['FIXED_WEIGHT'] = 1 / (filteredRecords['TRANSFORMED_VARIANCE'])
    filteredRecords['FIXED_WEIGHTED_PREVALENCE'] = filteredRecords['TRANSFORMED_PREVALENCE'] * filteredRecords['FIXED_WEIGHT']
    filteredRecords['INVERSE_DENOMINATOR'] = 1 / filteredRecords['DENOMINATOR']
    
    N_sum = sum(filteredRecords['DENOMINATOR'])
    fixed_weight_sum = sum(filteredRecords['FIXED_WEIGHT'])
    fixed_weighted_prevalence_sum = sum(filteredRecords['FIXED_WEIGHTED_PREVALENCE'])
    variance_sum = sum(filteredRecords['TRANSFORMED_VARIANCE'])
    inverse_n_sum = sum(filteredRecords['INVERSE_DENOMINATOR'])
    
    trans_pooled_prevalence = fixed_weighted_prevalence_sum / fixed_weight_sum
    trans_conf_inter = [trans_pooled_prevalence - Z_975 * sqrt(variance_sum),
                       trans_pooled_prevalence + Z_975 * sqrt(variance_sum)]
    
    overall_n = n_studies / inverse_n_sum
    
    if not random_effects:
        pooled_prevalence = backTransformPrevalence(trans_pooled_prevalence, overall_n, transformation_method)
        conf_inter = [backTransformPrevalence(i, overall_n, transformation_method) for i in trans_conf_inter]
        conf_inter = [max(conf_inter[0], 0), min(conf_inter[1], 1)]
        error = [abs(pooled_prevalence - i) for i in conf_inter]
        return {
            'seroprevalence_percent': pooled_prevalence * 100,
            'error_percent': [i * 100 for i in error],
            'total_N': N_sum,
            'n_studies': n_studies
        }
    else:
        filteredRecords['TRANSFORMED_POOLED_PREVALENCE'] = trans_pooled_prevalence
        tau = calcBetweenStudyVariance(filteredRecords)
        filteredRecords['TRANSFORMED_TOTAL_VARIANCE'] = filteredRecords['TRANSFORMED_VARIANCE'] + tau
        filteredRecords['RANDOM_WEIGHT'] = 1 / filteredRecords['TRANSFORMED_TOTAL_VARIANCE']
        filteredRecords['RANDOM_WEIGHT_PREVALENCE'] = filteredRecords['TRANSFORMED_PREVALENCE'] * filteredRecords['RANDOM_WEIGHT']
        
        random_weight_sum = sum(filteredRecords['RANDOM_WEIGHT'])
        random_weighted_prevalence_sum = sum(filteredRecords['RANDOM_WEIGHT_PREVALENCE'])
        variance_sum += tau
        trans_pooled_prevalence = random_weighted_prevalence_sum / random_weight_sum
        trans_conf_inter = [trans_pooled_prevalence - Z_975 * sqrt(variance_sum),
                       trans_pooled_prevalence + Z_975 * sqrt(variance_sum)]
        
        pooled_prevalence = backTransformPrevalence(trans_pooled_prevalence, overall_n, transformation_method)
        conf_inter = [backTransformPrevalence(i, overall_n, transformation_method) for i in trans_conf_inter]
        conf_inter = [max(conf_inter[0], 0), min(conf_inter[1], 1)]
        error = [abs(pooled_prevalence - i) for i in conf_inter]
        return {
            'seroprevalence_percent': pooled_prevalence * 100,
            'error_percent': [i * 100 for i in error],
            'total_N': N_sum,
            'n_studies': n_studies
        }
        

In [95]:
def groupBy(records, factor):
    options = {item for sublist in data[factor] for item in sublist}
    records.dropna(subset = [factor], inplace = True)
    return {name: records[records[factor].apply(lambda x: name in x)] for name in options}

{name: metaprop(records, random_effects = False) for name, records in groupBy(data, 'COUNTRY').items()}

{'Luxembourg': {'seroprevalence_percent': 5.589547571750364,
  'error_percent': [1.3967521143821104, 1.5837837015461598],
  'total_N': 3640.0,
  'n_studies': 2},
 'Switzerland': {'seroprevalence_percent': 4.4824110586886565,
  'error_percent': [3.821899164576098, 6.969743422554053],
  'total_N': 8338.0,
  'n_studies': 10},
 'Belgium': {'seroprevalence_percent': 4.8201414069825255,
  'error_percent': [1.1662266083585116, 1.3183042932171707],
  'total_N': 10363.0,
  'n_studies': 3},
 'Spain': {'seroprevalence_percent': 5.577107376851876,
  'error_percent': [3.301633713599772, 4.641140708461721],
  'total_N': 65741.0,
  'n_studies': 6},
 'Sweden': {'seroprevalence_percent': 10.807290869262303,
  'error_percent': [3.075135323122108, 3.517259101072284],
  'total_N': 6595.0,
  'n_studies': 3},
 'Brazil': {'seroprevalence_percent': 1.0498319162757008,
  'error_percent': [0.8512748925622204, 1.510673109892019],
  'total_N': 39666.0,
  'n_studies': 6},
 'Italy': {'seroprevalence_percent': 15.55

In [92]:
metaprop(data, random_effects = False)

{'seroprevalence_percent': 4.502462169737742,
 'error_percent': [4.502462169737742, 23.597591399976213],
 'total_N': 599663.0,
 'n_studies': 117}